In [4]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format







In [5]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-06-21'
start_date = '2016-01-01'
# Descargar el dataframe
intc_df = yf.download('INTC',start_date, today).reset_index()

intc_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1873,2023-06-13,$33.77,$33.95,$33.07,$33.91,$33.91,51990000
1874,2023-06-14,$33.87,$35.60,$33.80,$35.58,$35.58,74404300
1875,2023-06-15,$35.28,$36.42,$35.22,$35.82,$35.82,67676300
1876,2023-06-16,$36.36,$36.80,$35.60,$36.37,$36.37,110188500
1877,2023-06-20,$36.77,$37.11,$34.72,$35.00,$35.00,65882800


In [6]:
intc_df[["Date","Open"]]

,Date,Open
0,2016-01-04,$33.88
1,2016-01-05,$33.96
2,2016-01-06,$33.25
3,2016-01-07,$32.28
4,2016-01-08,$32.09
...,...,...
1873,2023-06-13,$33.77
1874,2023-06-14,$33.87
1875,2023-06-15,$35.28
1876,2023-06-16,$36.36


In [7]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del BTC en ese momento
df = intc_df[["Date","Open"]]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds",
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(columns=new_names)

In [8]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [9]:
df.tail()

,ds,y
1873,2023-06-13,$33.77
1874,2023-06-14,$33.87
1875,2023-06-15,$35.28
1876,2023-06-16,$36.36
1877,2023-06-20,$36.77


In [10]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = ""
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

In [11]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"

m = Prophet()
m.fit(df)

11:02:44 - cmdstanpy - INFO - Chain [1] start processing
11:02:45 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
#TAREA: Entrena el modelo m con el dataframe de datos df
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2238,2024-06-15
2239,2024-06-16
2240,2024-06-17
2241,2024-06-18
2242,2024-06-19


In [13]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = df['ds']
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2238,2024-06-15
2239,2024-06-16
2240,2024-06-17
2241,2024-06-18
2242,2024-06-19


In [14]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

# future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future = future[ future['ds'].dt.dayofweek < 5 ]

In [15]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)


In [16]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2134,2024-06-13,$4.95,$-4.23,$14.48
2135,2024-06-14,$4.63,$-4.51,$14.13
2136,2024-06-17,$3.83,$-5.74,$13.24
2137,2024-06-18,$3.76,$-6.06,$13.53
2138,2024-06-19,$3.37,$-6.05,$13.47


In [17]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)